# Part one

In [1]:
# import each library for assignment

import numpy as np
import cmath
import matplotlib.pyplot as plt
from cvxopt import matrix
from cvxopt import solvers

In [2]:
# load and process data

def data_processing():

    sample_X = np.loadtxt('./PA-1-data-text/polydata_data_sampx.txt')
    sample_Y = np.loadtxt('./PA-1-data-text/polydata_data_sampy.txt')
    poly_X = np.loadtxt('./PA-1-data-text/polydata_data_polyx.txt')
    poly_Y = np.loadtxt('./PA-1-data-text/polydata_data_polyy.txt')

    true_theta = np.loadtxt('./PA-1-data-text/polydata_data_thtrue.txt')

    sample_X = sample_X.reshape(1, len(sample_X))
    sample_Y = sample_Y.reshape(len(sample_Y), 1)

    poly_X = poly_X.reshape(1, len(poly_X))
    poly_Y = poly_Y.reshape(len(poly_Y), 1)

    # print(sample_X.shape)
    return sample_X, sample_Y, poly_X, poly_Y

# generate Phi(x).
def generate_Phix(X, K):
    return np.array( [ X**k for k in range(K+1)]).reshape(K+1, 1)

# generate Big Phi matrix dimen is (K, N).
def generate_Phi_mat(Sample, K):
    return np.array( [generate_Phix(x, K) for x in Sample[0]])[:,:, (Sample.shape[0]-1)].T

# Do the prediction though the fixed Theta.
def pred_polyY(polyX, Theta, K):
    Phi_mat_T = generate_Phi_mat(polyX, K).T
    pred_Y = np.dot(Phi_mat_T, Theta)
    return pred_Y

def least_squares_Theta(SampleX, SampleY, K):
    Phi_mat = generate_Phi_mat(Sample=SampleX, K=K)
    Theta = np.dot(np.linalg.inv(np.dot(Phi_mat, Phi_mat.T)), np.dot(Phi_mat, SampleY))
    return Theta

def Regular_LS_Theta(SampleX, SampleY, K, lbda=0.1):
    Phi_mat = generate_Phi_mat(Sample=SampleX, K=K)
    Theta = np.dot(np.linalg.inv(np.dot(Phi_mat, Phi_mat.T) + lbda * np.ones((K+1, K+1))), np.dot(Phi_mat, SampleY))
    return Theta

def L1_LS_Theta(SampleX, SampleY, K, lbda=1):
    # To solve Lasso, construct the matrix: 
    # X dimen:(2k, 1), H dimen:(2K, 2K), f dimen:(2K, 1), G based on constraint dimen: (2K, 2K), h (2K， 1) matrix
    Phi_mat = generate_Phi_mat(SampleX, K)
    Phi_square = np.dot(Phi_mat, Phi_mat.T)
    Phi_y = np.dot(Phi_mat, SampleY)
    H_left = np.concatenate((Phi_square, -Phi_square), axis=0)
    H = np.concatenate( (H_left, -H_left), axis=1 )
    f = lbda*np.ones((2*K, 1)) - np.concatenate( (Phi_y, -Phi_y), axis=0 )
    G = - np.identity(len(H))
    h = np.zeros( (len(H), 1) )
    Lasso_theta = solvers.qp(P=matrix(H), q=matrix(f), G=matrix(G), h=matrix(h))['x']
    
    # Lasso_theta is  [theta+, theta-]T, to get the theta = theta+ - theta-

    Theta = Lasso_theta[:K] - Lasso_theta[K:]
    

x, y, b, d = data_processing()
theta = L1_LS_Theta(x,y, 5)
pred = pred_polyY(b, theta, 5)
print(b.shape)
plt.scatter(x[0], y)
plt.plot(b[0], pred, 'b.')
plt.show()

NameError: name 'sample_X' is not defined